This case study uses VTK to build a simple, interactive visualizer for polydata. First, we are going to set up a class that represents each layer of the polydata itself in VTK.

In [20]:
import vtk

class ssLayer:
    def __init__ (self, interactor, data):
        self.interactor = interactor
        self.data = data
        
        self.properties = vtk.vtkProperty()
        
        self.mapper = vtk.vtkPolyDataMapper()
        self.mapper.SetInputData(self.data)
        
        self.actor = vtk.vtkActor()
        self.actor.SetMapper(self.mapper)
        self.actor.SetProperty(self.properties)
    
        self.box = vtk.vtkBoxWidget()
        self.box.SetInteractor(self.interactor)
        self.box.SetPlaceFactor(1.00)
        self.box.SetInputData(self.data)
        self.box.InsideOutOn()
        self.box.PlaceWidget()
        self.box.EnabledOff()
        
        self.planes = vtk.vtkPlanes()
        self.box.GetPlanes(self.planes)
        
        self.mapper.SetClippingPlanes(self.planes)
        
        self.box.AddObserver("InteractionEvent", self.boxClip)
        
    def boxClip (self, widget, event_string):
        self.box, self.mapper
        self.box.GetPlanes(self.planes)
        self.mapper.SetClippingPlanes(self.planes)

Above, what you see is a class that handles each layer of the visualization in VTK. Each layer has its own box and can be independently clipped via the callback boxClip member function. The mapper interacts with the actor to set properties on the data (such as the transfer function in volume rendering, or in this case properties like color and opacity). Now, let's get our first polydata object. First, we have to build the renderer and set some basic properties so we can tell each layer which render it belongs to. Then we simply add the actor to the renderer and render the window!

Now, let's instantiate a layer for this data. First, we have to build the renderer and set some basic properties so we can tell each layer which render it belongs to. Then we simply add the actor to the renderer and render the window!

In [21]:
renderer = vtk.vtkRenderer()
renderWindow = vtk.vtkRenderWindow()
renderWindow.AddRenderer(renderer)
renderWindowInteractor = vtk.vtkRenderWindowInteractor()
renderWindowInteractor.SetRenderWindow(renderWindow)
        
interactionStyle = vtk.vtkInteractorStyleTrackballCamera()
renderWindowInteractor.SetInteractorStyle(interactionStyle)

reader = vtk.vtkPolyDataReader()
reader.SetFileName("data/vtk/Subcortical_Structure__1_1.vtk")
reader.Update()
data = reader.GetOutput()

dataLayerA = ssLayer(renderWindowInteractor, data)

renderer.AddActor(dataLayerA.actor)
renderWindow.Render()
renderWindowInteractor.Start()

Success! This in and of itself maybe isn't all that exciting. It would be much better to be able to use multiple structures and interact with them independently. So let's add in a couple of data sets then. Then, 

In [22]:
import webcolors

renderer = vtk.vtkRenderer()
renderWindow = vtk.vtkRenderWindow()
renderWindow.AddRenderer(renderer)
renderWindowInteractor = vtk.vtkRenderWindowInteractor()
renderWindowInteractor.SetRenderWindow(renderWindow)
        
interactionStyle = vtk.vtkInteractorStyleTrackballCamera()
renderWindowInteractor.SetInteractorStyle(interactionStyle)

reader = vtk.vtkPolyDataReader()
reader.SetFileName("data/vtk/Subcortical_Structure__1_1.vtk")
reader.Update()
data = reader.GetOutput()

dataLayerA = ssLayer(renderWindowInteractor, data)
dataLayerA.properties.SetColor(webcolors.name_to_rgb('red'))
dataLayerA.properties.SetOpacity(0.4)
dataLayerA.box.EnabledOn()
renderer.AddActor(dataLayerA.actor)

reader = vtk.vtkPolyDataReader()
reader.SetFileName("data/vtk/Subcortical_Structure__2_2.vtk")
reader.Update()
data = reader.GetOutput()

dataLayerB = ssLayer(renderWindowInteractor, data)
dataLayerB.properties.SetColor(webcolors.name_to_rgb('green'))
dataLayerB.properties.SetOpacity(0.4)
renderer.AddActor(dataLayerB.actor)

reader = vtk.vtkPolyDataReader()
reader.SetFileName("data/vtk/Subcortical_Structure__3_3.vtk")
reader.Update()
data = reader.GetOutput()

dataLayerC = ssLayer(renderWindowInteractor, data)
renderer.AddActor(dataLayerC.actor)

renderWindow.Render()
renderWindowInteractor.Start()